In [8]:
import tqdm
import torch
from dataloader import Patches, DatasetType, ProposalsEval
from torch.utils.data import DataLoader
import os
from networks import Architecture
from assert_gpu import assert_gpu
import torchvision
from mAP import calc_AP
from evalution import nms
device = assert_gpu()
import numpy as np

In [9]:
    modelpath = os.path.join("models","restfull_netowrk.pdf")
    architecture = Architecture.from_string("resnet18")
    
    model = architecture.create_network()
    model.load_state_dict(torch.load(modelpath))
    model.to(device)

ResNet18(
  (net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [ ]:
    proposal_set = ProposalsEval(root_dir= "data_wastedetection",ds_type = DatasetType.train)
    img_loader = DataLoader(proposal_set, batch_size=1, shuffle=False, num_workers=1)

    cat_scores = [torch.tensor([]) for _ in range(59)]
    cat_trues = [torch.tensor([]) for _ in range(59)]
    for img_num, (proposals, image, true_bb, true_cat) in enumerate(img_loader):
        print('Working on image ', img_num)

        image = image.squeeze()
        proposals = proposals.squeeze()
    
        batch_size = 64
        patch_set = Patches(image, proposals)
        patch_loader = DataLoader(patch_set, batch_size=batch_size, shuffle=False, num_workers=1)
    
        predictions = []
        for patch in patch_loader:
            patch = patch.to(device).float()
            predictions.append(model(patch))

        predictions = torch.cat(predictions).to('cpu')
        predictions = torch.nn.functional.softmax(predictions)

        confidscore, catargmax = torch.max(predictions, dim=1)

        # remove backgrounds
        confidscore = confidscore[catargmax!=59]
        proposals = proposals[catargmax!=59]
        catargmax = catargmax[catargmax!=59]

        # change proposals from X,Y,W,H to X,Y,X2,Y2
        proposals[:,2] =  proposals[:,2] + proposals[:,0]
        proposals[:,3] = proposals[:,3] + proposals[:,1]

        for cat in range(59):
            # filter by category and sort by confidence score
            cat_confidence = confidscore[catargmax==cat]
            order = torch.argsort(cat_confidence, descending=True)
            cat_proposals = proposals[catargmax==cat][order]
            cat_confidence[order]

            # remove unneeded bb with NMS
            proposal_inds  = nms(cat_proposals,cat_confidence,iou_threshold=0.5)
            cat_proposals  = cat_proposals[proposal_inds]
            cat_confidence = cat_confidence[proposal_inds]
            
            

            # check which bb are true detections
            iou_threshold = 0.5
            true = true_bb[true_cat==cat+1]
            if len(true)==0:
                cat_true = torch.tensor([0]*len(cat_proposals))
            else:
                if len(cat_confidence)>0:
                    iou_bb = torchvision.ops.box_iou(cat_proposals, true_bb[true_cat==cat+1])
                    cat_true = iou_bb.max(1)[0]>iou_threshold
                    missed = max(len(true) - sum(cat_true),0)
                else:
                    missed = len(true)
                    cat_true = torch.tensor([],dtype=int)
                
                cat_scores[cat] = torch.cat([cat_scores[cat],torch.tensor([0]*missed)])
                cat_trues[cat] = torch.cat([cat_trues[cat],torch.tensor([1]*missed)])

            cat_scores[cat] = torch.cat([cat_scores[cat],cat_confidence])
            cat_trues[cat] = torch.cat([cat_trues[cat],cat_true])
            
    

Number of images in dataset: 1000
Working on image  0
Working on image  1
Working on image  2
Working on image  3
Working on image  4
Working on image  5
Working on image  6
Working on image  7
Working on image  8
Working on image  9
Working on image  10
Working on image  11
Working on image  12
Working on image  13
Working on image  14
Working on image  15
Working on image  16
Working on image  17
Working on image  18
Working on image  19
Working on image  20
Working on image  21
Working on image  22
Working on image  23
Working on image  24
Working on image  25
Working on image  26
Working on image  27
Working on image  28
Working on image  29
Working on image  30
Working on image  31
Working on image  32
Working on image  33


In [7]:
calc_AP(cat_scores, cat_trues)

Calculated mAP:  0.0


[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]